In [14]:
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow import keras
import glob
import pathlib
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.callbacks import LearningRateScheduler
from keras.utils import load_img, img_to_array
import os as os 

In [24]:
#Define the shape for the test inputs

image_size = (100, 100)
input_shape = (None,) + image_size + (3,)
batch_size = 32

In [16]:
#Loading the test data
test_dir = pathlib.Path('Data/test')

In [25]:
#Creating the test dataset

test_ds = image_dataset_from_directory(
    test_dir, 
    image_size=image_size,
    batch_size=batch_size,
    label_mode='categorical',
    seed = 42,
)

Found 539 files belonging to 3 classes.


In [33]:
#Loading trained model
final_model = keras.models.load_model('increased_shape.keras')

final_model.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_11 (Sequential)  (None, 100, 100, 3)       0         
                                                                 
 conv2d_40 (Conv2D)          (None, 100, 100, 64)      1792      
                                                                 
 batch_normalization_40 (Bat  (None, 100, 100, 64)     256       
 chNormalization)                                                
                                                                 
 re_lu_40 (ReLU)             (None, 100, 100, 64)      0         
                                                                 
 max_pooling2d_40 (MaxPoolin  (None, 50, 50, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_41 (Conv2D)          (None, 50, 50, 128)     

In [34]:
#Evaluating model on test set

final_model.evaluate(test_ds)

17/17 [==============================] - 2s 111ms/step - loss: 0.4632 - accuracy: 0.8571


[0.46320730447769165, 0.8571428656578064]

In [35]:
#Extract class names for sample predictions

class_names = test_ds.class_names

print(class_names)

['crack', 'missing-head', 'paint-off']


In [36]:
#Loading the specified test samples

img_paths = ['Data/test/crack/test_crack.jpg', 
            'Data/test/missing-head/test_missinghead.jpg',
            'Data/test/paint-off/test_paintoff.jpg' 
]

test_imgs = []

for img in img_paths:
    img_array = img_to_array(load_img(img, target_size=image_size))
    img_batch = np.expand_dims(img_array, axis=0)
    test_imgs.append(img_batch)

In [37]:
#Making predictions on the sample images

for img in test_imgs:
    print(final_model.predict(img))

1/1 [==============================] - 0s 200ms/step
[[0.6489233  0.07001609 0.28106055]]
1/1 [==============================] - 0s 32ms/step
[[8.6857661e-05 9.9990106e-01 1.1982223e-05]]
1/1 [==============================] - 0s 31ms/step
[[0.02560018 0.00960403 0.96479577]]
